## Importing the Packages

In [137]:
import numpy as np
import pandas as pd

In [138]:
np.set_printoptions(suppress=True, linewidth=100, precision=2)
# stop numpy from usimg scientific notation
# extend the number of chars we fit in a single line of output to 100
# only display the first two digits after the decimal point

## Importing the Data

In [139]:
raw_data_np = np.genfromtxt("loan-data.csv", delimiter=';', skip_header=1, autostrip=True)
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Incomplete Data

In [140]:
np.isnan(raw_data_np).sum()

88005

In [141]:
temporary_fill = np.nanmax(raw_data_np) + 1
temporary_mean = np.nanmean(raw_data_np, axis=0)

<ipython-input-141-2ab24ed5c8e4>:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data_np, axis=0)


In [142]:
temporary_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [143]:
temp_stats = np.array([np.nanmin(raw_data_np, axis=0),
                        temporary_mean,
                        np.nanmax(raw_data_np, axis=0)])

<ipython-input-143-ab3741d59e6e>:1: RuntimeWarning: All-NaN slice encountered
  temp_stats = np.array([np.nanmin(raw_data_np, axis=0),
<ipython-input-143-ab3741d59e6e>:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data_np, axis=0)])


In [144]:
temp_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

## Splitting the Dataset

### Splitting the Columns

In [145]:
col_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
col_strings
# get the indices of the cols that only contain strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [146]:
col_num = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
col_num

array([ 0,  2,  4,  6,  7, 13])

### Re-importing the Dataset

In [147]:
# the string dataset
loan_data_str = np.genfromtxt(
    "loan-data.csv", 
    delimiter=';', 
    skip_header=1, 
    autostrip=True, 
    usecols=col_strings, 
    dtype=str)
loan_data_str

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [148]:
# the numeric dataset
loan_data_numeric = np.genfromtxt(
    "loan-data.csv", 
    delimiter=';', 
    skip_header=1, 
    autostrip=True, 
    usecols=col_num, 
    filling_values=temporary_fill)
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

### The Names of the Columns

In [149]:
header_full = np.genfromtxt(
    "loan-data.csv", 
    delimiter=';', 
    autostrip=True, 
    skip_footer=raw_data_np.shape[0], #ignore all rows after the header
    dtype=str)
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [150]:
# split the headers into two variables, each one holding the names of 
# the colums that contain str data or num data
header_str = header_full[col_strings]
header_str

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [151]:
header_num = header_full[col_num]
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Creating Checkpoints:

In [152]:
"""
checkpoints:
places throughout the code where we store a copy of our dataset(or parts of it)
we want to avoid losing a lot of progress
a reliable practice when we need to clean or preprocess many parts of a dataset
"""

'\ncheckpoints:\nplaces throughout the code where we store a copy of our dataset(or parts of it)\nwe want to avoid losing a lot of progress\na reliable practice when we need to clean or preprocess many parts of a dataset\n'

In [153]:
def checkpoint(filename, checkpoint_header, checkpoint_data):
    np.savez(filename, header=checkpoint_header, data=checkpoint_data)
    checkpoint_var = np.load(filename + ".npz")
    return(checkpoint_var)

In [154]:
checkpoint_test = checkpoint('checkpoint-test', header_str, loan_data_str)


In [155]:
checkpoint_test['header']

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [156]:
checkpoint_test['data']

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [157]:
np.array_equal(checkpoint_test['header'], loan_data_str)

False

In [158]:
np.array_equal(checkpoint_test['data'], loan_data_str)

True

## Manipulating String Columns

In [159]:
header_str # names of the cols that contain only str values

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [160]:
header_str[0]

'issue_d'

In [161]:
header_str[0] = "issue_date"

In [162]:
loan_data_str

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [163]:
header_str

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

### Issue Date

In [164]:
np.unique(loan_data_str[:,0])
# display only the first column of dataset

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [165]:
# remove the year
loan_data_str[:,0] = np.chararray.strip(loan_data_str[:,0], "-15")

In [166]:
np.unique(loan_data_str[:,0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

In [ ]:
"""
In Analysis:
represent month values as integers
store the data using less memory 
a more easy to follow order of months
"""

In [167]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

for i in range(13):
        loan_data_str[:,0] = np.where(loan_data_str[:,0] == months[i],
                                          i,
                                          loan_data_str[:,0])

In [168]:
np.unique(loan_data_str[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Loan Status

In [169]:
header_str
# display columns to see what second column is called

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [172]:
np.unique(loan_data_str[:,1]) 
# examine it more closely

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [173]:
np.unique(loan_data_str[:,1]).size

9

In [174]:
status_bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

In [175]:
loan_data_str[:,1] = np.where(np.isin(loan_data_str[:,1], status_bad), 0,1)

In [176]:
np.unique(loan_data_str[:,1])

array(['0', '1'], dtype='<U69')

### Term

In [177]:
header_str

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [178]:
np.unique(loan_data_str[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

In [179]:
loan_data_str[:,2] = np.chararray.strip(loan_data_str[:,2], " months")
loan_data_str[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [182]:
header_str[2] = "term_months"

In [183]:
header_str

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [184]:
loan_data_str[:,2] = np.where(loan_data_str[:,2] == '', '60', loan_data_str[:,2])
loan_data_str[:,2]
# 60 is chosen to replace the empty strings beause this number represents the worst case 60 = 5 years

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [185]:
np.unique(loan_data_str[:,2])

array(['36', '60'], dtype='<U69')

### Grade and Subgrade

In [186]:
header_str

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [187]:
loan_data_str[:,3]

array(['C', 'A', 'B', ..., 'A', 'D', 'A'], dtype='<U69')

In [188]:
np.unique(loan_data_str[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [189]:
np.unique(loan_data_str[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

#### Filling Sub Grade

In [191]:
for i in np.unique(loan_data_str[:,3])[1:]: #go thru all the unique grades after the first one
    # check if the subgrade is missing, check if the grade for that row == grade for current iteration
    loan_data_str[:,4] = np.where((loan_data_str[:,4] == '') & (loan_data_str[:,3] == i), 
                                      i + '5',
                                      loan_data_str[:,4])

In [192]:
np.unique(loan_data_str[:,4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))

In [193]:
# create a new category lower than G5
loan_data_str[:,4] = np.where(loan_data_str[:,4] == '', 'H1', loan_data_str[:,4])

In [194]:
np.unique(loan_data_str[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

#### Removing Grade

In [195]:
loan_data_str = np.delete(loan_data_str, 3, axis=1)

In [196]:
loan_data_str[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [197]:
header_str = np.delete(header_str, 3)

In [198]:
header_str[3]

'sub_grade'

In [199]:
header_str

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

#### Converting Sub Grade

In [200]:
np.unique(loan_data_str[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [202]:
keys = list(np.unique(loan_data_str[:,3]))                         
values = list(range(1, np.unique(loan_data_str[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))

In [203]:
dict_sub_grade

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

In [206]:
for i in np.unique(loan_data_str[:,3]):
        loan_data_str[:,3] = np.where(loan_data_str[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_str[:,3])

In [207]:
np.unique(loan_data_str[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Verification Status

In [208]:
header_str

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [211]:
header_str[4]

'verification_status'

In [212]:
loan_data_str[:,4]

array(['Verified', 'Source Verified', 'Verified', ..., 'Source Verified', 'Source Verified', ''],
      dtype='<U69')

In [213]:
np.unique(loan_data_str[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [214]:
loan_data_str[:,4] = np.where((loan_data_str[:,4] == '') | (loan_data_str[:,4] == 'Not Verified'), 0, 1)

In [215]:
np.unique(loan_data_str[:,4])

array(['0', '1'], dtype='<U69')

### URL

In [216]:
loan_data_str

array([['5', '1', '36', ..., '1',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['0', '1', '36', ..., '1',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['9', '1', '36', ..., '1',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['6', '1', '36', ..., '1',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['4', '1', '36', ..., '1',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['12', '1', '36', ..., '0',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [217]:
loan_data_str[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [218]:
np.chararray.strip(loan_data_str[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

chararray(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'],
          dtype='<U69')

In [219]:
loan_data_str[:,5] = np.chararray.strip(loan_data_str[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [220]:
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [221]:
loan_data_numeric[:,0].astype(dtype= np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], dtype=int32)

In [222]:
loan_data_str[:,5].astype(dtype=np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], dtype=int32)

In [223]:
np.array_equal(
    loan_data_numeric[:,0].astype(dtype= np.int32), 
    loan_data_str[:,5].astype(dtype=np.int32)
    )

True

In [224]:
loan_data_str = np.delete(loan_data_str, 5, axis=1)
header_str = np.delete(header_str, 5)

In [225]:
loan_data_str[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [226]:
header_str

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [227]:
loan_data_numeric[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

In [228]:
header_num

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

### State Address

In [229]:
header_str

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [230]:
header_str[5]

'addr_state'

In [231]:
header_str[5] = "state_address"

In [232]:
states_names, states_count = np.unique(loan_data_str[:,5], return_counts=True)
states_count_sorted = np.argsort(-states_count) # - sign because we are sorting in decreasing order
states_names[states_count_sorted], states_count[states_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10]))

In [233]:
loan_data_str[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [238]:
loan_data_str[:,5].size

10000

In [234]:
loan_data_str[:,5] = np.where(
    loan_data_str[:,5] == '',
    0,
    loan_data_str[:,5]
)

In [235]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [236]:
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_west), 1, loan_data_str[:,5])
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_south), 2, loan_data_str[:,5])
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_midwest), 3, loan_data_str[:,5])
loan_data_str[:,5] = np.where(np.isin(loan_data_str[:,5], states_east), 4, loan_data_str[:,5])

In [237]:
np.unique(loan_data_str[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

## Converting to Numbers

In [239]:
loan_data_str

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [241]:
loan_data_str = loan_data_str.astype(int)

In [242]:
loan_data_str

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

### Checkpoint 1: Strings

## Manipulating Numeric Columns

### Substitute "Filler" Values

#### ID

#### Temporary Stats

#### Funded Amount

In [ ]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill, 
                                  temporary_stats[0, columns_numeric[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

#### Loaned Amount, Interest Rate, Total Payment, Installment

### Currency Change

#### The Exchange Rate

In [ ]:

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    


#### From USD to EUR

In [ ]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

#### Expanding the header

In [ ]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [ ]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

### Interest Rate

### Checkpoint 2: Numeric

In [ ]:
checkpoint_numeric['header'], checkpoint_numeric['data']

## Creating the "Complete" Dataset

## Sorting the New Dataset

## Storing the New Dataset